Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage 
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

## Download Dataset
First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.


In [7]:
url = 'http://commondatastorage.googleapis.com/books1000/'

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    if force or not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

#train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
#test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)
# IOError: [Errno socket error] [Errno 32] Broken pipe

# 手动下载的.....

### 解压数据
Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [12]:
num_classes = 10 # A-J total 10 classes labels
np.random.seed(133)

data_path = "../data/" # 数据下载在 data 文件夹下
train_filename = 'notMNIST_large.tar.gz'
test_filename = 'notMNIST_small.tar.gz'

def maybe_extract(filename, force=False):
    """extract gzip file"""
    root = os.path.splitext(os.path.splitext(data_path + filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root) and not force:
    # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(data_path + filename)
        sys.stdout.flush()
        tar.extractall(path = data_path) # 解压
        tar.close()
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))
                    if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders, one per class. Found %d instead.' % (
                num_classes, len(data_folders)))
    print(data_folders)
    return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for ../data/notMNIST_large. This may take a while. Please wait.
['../data/notMNIST_large/A', '../data/notMNIST_large/B', '../data/notMNIST_large/C', '../data/notMNIST_large/D', '../data/notMNIST_large/E', '../data/notMNIST_large/F', '../data/notMNIST_large/G', '../data/notMNIST_large/H', '../data/notMNIST_large/I', '../data/notMNIST_large/J']
Extracting data for ../data/notMNIST_small. This may take a while. Please wait.
['../data/notMNIST_small/A', '../data/notMNIST_small/B', '../data/notMNIST_small/C', '../data/notMNIST_small/D', '../data/notMNIST_small/E', '../data/notMNIST_small/F', '../data/notMNIST_small/G', '../data/notMNIST_small/H', '../data/notMNIST_small/I', '../data/notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package `IPython.display`.

---

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

- 分批储存数据, 再 merge, 以免 memory 不够
- image -> (index, x, y) array
- normalized to mean = 0, std = ~0.5
- skip not readable data

In [13]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
    """Load the data for a single letter label.""" # 一次只读取 一个字母的 folder
    image_files = os.listdir(folder) # image files
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size), # shape = (图片数量, 图片长, 宽)
                         dtype=np.float32)
    print(folder)
    for image_index, image in enumerate(image_files):
        image_file = os.path.join(folder, image) #对一个图片
        try: 
            image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth # normallized? 
            # ndimage.imread: Read an image from a file as an array.
            # return: The different colour bands/channels are stored in the third dimension, 
            # such that a grey-image is MxN, an RGB-image MxNx3 and an RGBA-image MxNx4.
            if image_data.shape != (image_size, image_size):
                raise Exception('Unexpected image shape: %s' % str(image_data.shape))
            dataset[image_index, :, :] = image_data #存入 dataset, 加上 index
        except IOError as e: # 读取不了的图片就 skip
            print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
    num_images = image_index + 1 # 图片数量
    dataset = dataset[0:num_images, :, :]
    if num_images < min_num_images: # 少于一定数量
        raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
    print('Full dataset tensor:', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))
    return dataset

In [14]:
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
    """dataset 转为 pickle 存下来"""
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
        if os.path.exists(set_filename) and not force:
            # You may override by setting force=True.
            print('%s already present - Skipping pickling.' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_letter(folder, min_num_images_per_class)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)
  
    return dataset_names

In [15]:
train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling ../data/notMNIST_large/A.pickle.
../data/notMNIST_large/A
Could not read: ../data/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : cannot identify image file '../data/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png' - it's ok, skipping.
Could not read: ../data/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : cannot identify image file '../data/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png' - it's ok, skipping.
Could not read: ../data/notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : cannot identify image file '../data/notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png' - it's ok, skipping.
Full dataset tensor: (52912, 28, 28)
Mean: -0.128243
Standard deviation: 0.443109
Pickling ../data/notMNIST_large/B.pickle.
../data/notMNIST_large/B
Could not read: ../data/notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : cannot identify image file '../data/notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png' - it's ok, skipping.
Full dataset ten

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use `matplotlib.pyplot`.

---

train_datasets 和 test_datasets 是 pickle 文件名字的列表

In [23]:
with open(train_datasets[0],'rb') as f:
    train_A = pickle.load(f)

In [24]:
type(train_A)

numpy.ndarray

In [25]:
train_A.shape # 一共 52912 个图片

(52912, 28, 28)

In [31]:
train_folders[0]

'../data/notMNIST_large/A'

In [34]:
images_A = os.listdir(train_folders[0])
for imageName in images_A[:10]:
    display(Image(filename=train_folders[0]+'/'+imageName))

In [35]:
tmp_A = train_A[:100,:,:]

In [ ]:
# 如何plot????

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

各个 label 下的数据是平衡的?

---

In [37]:
for fi in range(len(train_datasets)):
    with open(train_datasets[fi],'rb') as f:
        train_class = pickle.load(f)
        print(train_datasets[fi]," with shape ",str(train_class.shape))

../data/notMNIST_large/A.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/B.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/C.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/D.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/E.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/F.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/G.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/H.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/I.pickle  with shape  (52912, 28, 28)
../data/notMNIST_large/J.pickle  with shape  (52911, 28, 28)


除了 J 少一个图片, 其他是一样的

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [38]:
def make_arrays(nb_rows, img_size):
    if nb_rows:
        dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
        labels = np.ndarray(nb_rows, dtype=np.int32)
    else:
        dataset, labels = None, None
    return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
    """merge datasets, train_size can be modified if memory limited"""
    num_classes = len(pickle_files) # 多少 pickles 就有多少 classes
    # 创建一个 validation dataset, 为了 hyperparameter 
    valid_dataset, valid_labels = make_arrays(valid_size, image_size)
    train_dataset, train_labels = make_arrays(train_size, image_size)
    vsize_per_class = valid_size // num_classes
    tsize_per_class = train_size // num_classes
    
    start_v, start_t = 0, 0
    end_v, end_t = vsize_per_class, tsize_per_class
    end_l = vsize_per_class+tsize_per_class
    for label, pickle_file in enumerate(pickle_files): # 在 file 中循环       
        try:
            with open(pickle_file, 'rb') as f:
                letter_set = pickle.load(f)
                # let's shuffle the letters to have random validation and training set
                np.random.shuffle(letter_set)
                if valid_dataset is not None: # valid dataset 如果非空
                    valid_letter = letter_set[:vsize_per_class, :, :]
                    valid_dataset[start_v:end_v, :, :] = valid_letter
                    valid_labels[start_v:end_v] = label
                    start_v += vsize_per_class
                    end_v += vsize_per_class
                # train 和 valid 应该没有 overlap   
                train_letter = letter_set[vsize_per_class:end_l, :, :]
                train_dataset[start_t:end_t, :, :] = train_letter
                train_labels[start_t:end_t] = label
                start_t += tsize_per_class
                end_t += tsize_per_class
        except Exception as e:
            print('Unable to process data from', pickle_file, ':', e)
            raise
    
    return valid_dataset, valid_labels, train_dataset, train_labels

In [39]:
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [40]:
def randomize(dataset, labels):# for shuffle 
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels


train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

什么叫 still good???

---

Finally, let's save the data for later reuse:

In [41]:
pickle_file = '../data/notMNIST.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'valid_dataset': valid_dataset,
        'valid_labels': valid_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,}
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [42]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800441


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.


检查 overlapping samples....怎么检查??

merge_dataset() 函数的方法, 应该 train 和 valid 并没有 overlapping啊???

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

对比图片是否一致, https://discussions.udacity.com/t/assignment-1-problem-5/45657/21
    
hashlib numpy等

In [50]:
overlap = []
for i in xrange(10000):
    for j in xrange(10000-i-1):
        if valid_dataset[i,:,:].__eq__(valid_dataset[j+i+1,:,:]).all():
            overlap.append(valid_dataset[i,:,:])

In [51]:
len(overlap)

4512

---
Problem 6
---------

Let's get an idea of what an off-the-shelf (现成的) classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [2]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    all_data = pickle.load(f)

In [3]:
train_dataset = all_data['train_dataset']
train_labels = all_data['train_labels']
valid_dataset = all_data['valid_dataset']
valid_labels = all_data['valid_labels']
test_dataset = all_data['test_dataset']
test_labels = all_data['test_labels']

In [4]:
clf = LogisticRegression()

In [5]:
num, width, height = train_dataset.shape

In [7]:
X = train_dataset.reshape(num, width*height)[:10000]

In [8]:
X.shape

(10000, 784)

In [9]:
y = train_labels[:10000]

In [10]:
clf.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
clf.score(X,y)

0.9022

In [15]:
num2, width, height = test_dataset.shape

In [16]:
test_X = test_dataset.reshape(num2, width*height)[:1000]

In [17]:
test_true_y = test_labels[:1000]

In [18]:
predicted_y = clf.predict(test_X)

In [22]:
correct = 0.0
for i in xrange(1000):
    if test_true_y[i] == predicted_y[i]:
        correct += 1

In [24]:
print("Accuracy: ", str(correct/1000))

Accuracy:  0.874
